# Further Idea

## 1. The labels that can be added 

As what I said in the notebook of '1. Identify the Breaking News Event', the pipeline of solving the task could be do **classification** then **clustering**.

To do the classification, I need the training data set which have the labels indicating whether the news articles are about major breaking news or not. The classifier need such training data to get the knowledge.

Another thing is that the current performance is still not that good, on the clustering work. The current solution is to use the word2vec trained on title to do clustering. There are some noise words in the corpus, influencing the performance. If could, some key words could be added into the data set (if possible). Key words could play a more important role in the representing the sentence/article. (although I know the keys words may coule be extracted using NLP tools).

## 2. Some other exploreing experiences during this challenge 

### doc2vec implemented by Gensim

`doc2vec` can be used to extract the sentence representations.

I tried to use doc2vec here. Since there is no good pre-trained `doc2vec` model that I can use. The training corpus could influence the performance of model a lot. I didn't spend time to train the `doc2vec`. While `spacy` supplies good pre-trained model and API, I used that in my answer of this challenge.

The args in doc2vec model:
- vector_size: the length of the vectors got at last
- window: the maximum distance between the current word to the predicted words
- alpha: the learning rate 
- min_alpha: Learning rate will linearly drop to min_alpha as training progresses.
- min_count: Ignore the words that total count lower than this.

see the doc for more detail:https://radimrehurek.com/gensim/models/doc2vec.html


In [4]:
# Doc2vec
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import logging
import pandas as pd

In [5]:
modelCorpus=pd.read_csv("data/news_corpus_for_doc2vec.csv") 
modelCorpus.head()
# The csv file was generated from "1. Identify the Breaking News Event" notebook.
# The current csv file is generated from shorten data set

,Words_Tokenized
0,lowestoft sea wall fall cyclist rescued friend
1,met gala game thrones feature week top comment...
2,boy dies prom day allergic reaction pollen cel...
3,paddy jackson return rugby yet proof misogyny ...
4,bafta awards stars prepare glitzy ceremony kil...


In [6]:
# Only need to uncomment this when I need to re-train this model.
max_epochs = 100
alpha = 0.025

documents = [TaggedDocument(content['Words_Tokenized'], [index]) for index, content in modelCorpus.iterrows()]
model = Doc2Vec(documents, vector_size=100,alpha=alpha, min_alpha=0.00025, window=3, min_count=5)
# model.build_vocab(documents)

for epoch in range(max_epochs):
    if not epoch%10:
        print('iteration {0}.. Doing training'.format(epoch))
    model.train(documents,total_examples=model.corpus_count,epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
print("training finished.")


iteration 0.. Doing training


/home/sa2y18/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  if sys.path[0] == '':


iteration 10.. Doing training
iteration 20.. Doing training
iteration 30.. Doing training
iteration 40.. Doing training
iteration 50.. Doing training
iteration 60.. Doing training
iteration 70.. Doing training
iteration 80.. Doing training
iteration 90.. Doing training
training finished.


In [ ]:
# Save or load the model
# model.save("doc2vecModel_corpus")
# model = Doc2Vec.load("doc2vecModel_corpus")

In [7]:
len(model.docvecs[0])

100

### BERT implemented by `pytorch_transformer`

During the challenge, I tried to use BERT on my solution. The innotion of BERT is applying the bidirectional training of Transformer (Build all based on Attention Mechanism).

`pytorch-transformer` supplies the BERT of `pytorch` version, and I can use the pre-trained model on my task. It's transfer learning and a head needs to be added for fine-tuninng. In the code repo, Hugging face building the net based on BERT for multiple NLP work, like the sentence classification, next sentence prediction, GLUE and NER task.  
-`pytorch-transformer` https://github.com/huggingface/pytorch-transformers


---
During task 1 of this challenge, I was trying to apply `BertForNextSentencePrediction` to tackle the problem. 

The idea is that to build corpus based on the news data set. See the title and description as the sentence pair of a specific news article. For a news article, the title and its description should have high similarity. `BertForNextSentencePrediction` can capture this relationship. I load the pre-trained model and only fine-tuned the head for my work.

After training the model, use the model to measure the similarity of each title (can be seen as the Distance). From this, I could be able to do the clustering.

Since I just learned the `BERT` and `Spacy` in these two days, I'm still on the stage of exercising demo and reading source code. Below code is the part of using `BERT` for the next sentence prediction.

See the code details of this on [Using BERT NextSentence Prediction for Task1](Using BERT NextSentence Prediction for Task1.ipynb)

## 3. Summary 

To summary for this whole project, I did some NLP tasks.

In task 1, I did some work about text cleaning, word vectors training and using DBSCAN to do the clustering. It works. though the performance is not very good.

In task 2, NER is the major thing I did. The final results is kind of terrible. I should do more training on the models if want to get better results.

I wanted to build some scripts to do the work. After finishing a small demo of the data set, I found the jupyter is a better choice for displaying. So the works are displayed by notebook.

And in the last, thanks for echobox, for supplying me a such good challenge to finish. It makes me learn a lot during tackling the challenge.